In [1]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import h5py

from matplotlib import animation
from matplotlib import cm
from mpl_toolkits import mplot3d
import os,sys
sys.path.append("/Users/ajc/Core/PostProcessData/PyScript")
import readdata as rd
import os
import cv2


In [3]:
#mesh_file='/Users/ajc/FUSE/ats-tests/ats-transport-subgrid-testsuite/tests/logical-surface-twoway/coweeta/test2'
path_cades='/Users/ajc/FUSE1/simulations/tests/ats-tests/ats-transport-subgrid-testsuite/tests/logical-surface-twoway/coweeta/Srun1A'
                           
infile = [os.path.join(path_cades,'visdump_surface_data.h5'), os.path.join(path_cades,'visdump_surface_mesh.h5') ]

mesh = h5py.File(infile[1],'r')
mesh_data = list(mesh['0']['Mesh']['Nodes'])

C_data = rd.GetSurfVarFromVis_All(infile[0],'surface-total_component_concentration',suffix='.cell.Tc-99')
Concentration = C_data['surface-total_component_concentration']
cycles = len(Concentration)
print (len(Concentration))

49
49


In [4]:
X_mesh = []
Y_mesh = []
Z_mesh = []

for m in mesh_data:#[:30]:
    X_mesh.append(m[0])
    Y_mesh.append(m[1])
    Z_mesh.append(m[2])
print (len(Concentration), len(Concentration[0]), len(Z_mesh))

def subgrid_mesh(mesh_sg_data):
    X_sgmesh,Y_sgmesh,Z_sgmesh=[],[],[]
    for m in mesh_sg_data:
        X_sgmesh.append(m[0])
        Y_sgmesh.append(m[1])
        Z_sgmesh.append(m[2])
    return X_sgmesh,Y_sgmesh,Z_sgmesh 

def subgrid_C(sgfile,varname_sg):
    sgC_data = rd.GetSurfVarFromVis_All(sgfile,varname_sg,suffix='.cell.Tc-99')
    sgConcentration = sgC_data[varname_sg]
    #print ('subgrid C: ',sgConcentration)
    return sgConcentration

49 986 986
49 986 986


In [2]:
path='/Users/ajc/Core/PostProcessData/2019/simulations/transport/Coweeta/Srun1A/'

In [6]:
#Write solute concentration images to a directory
%matplotlib tk

#fig = plt.figure()
#ax = fig.gca(projection='3d')

x_orig = X_mesh
y_orig = Y_mesh

centroid = []
thick = [1,1,1]
print (cycles)
count = 0
for k in range(0,cycles,1):
    #fig = plt.figure()#subplots(1,1,figsize=(8,4))
    fig = plt.figure(figsize=(8,4))#subplots(1,1,)
    ax = fig.gca(projection='3d',adjustable='box')
    
    y1 = y_orig
    y2 = [i + 0.000001 for i in y1]
    X = [x_orig,x_orig]
    Y = [y1,y2]
    
    z = np.ones(len(x_orig))*1

    Z = np.array([z,z])
    data = np.linspace(1,0,len(x_orig))
    Z_data = np.array([data,data])
    
    data_color = cm.jet(Z_data/np.amax(Z_data))
    #ran = [f*1 for f in flux_interpolate] #Discharge_stream
    ran = Concentration[k]
    ran = [r if r>0 else 1e-12 for r in ran]
    sc = ax.scatter(X_mesh,Y_mesh,z,marker=".",c=ran,cmap=cm.coolwarm,vmin=1e-04,vmax=.1,norm=matplotlib.colors.LogNorm())
    
    
    plt.colorbar(sc, label='Concentration [kg/m$^3$]',shrink=.96)#,aspect=10)
   
    #ax.set_zlim(0,0.0)
    #ax.set_zticks([])
    
    xmin, xmax = ax.get_xlim()
    ax.set_xticks(np.round(np.linspace(xmin, xmax, 5), 2))
    ax.set_xticklabels(np.round(np.linspace(xmin, xmax, 5), 2), rotation=45, fontsize=10)
    
    xmin, xmax = ax.get_ylim()
    ax.set_yticks(np.round(np.linspace(xmin, xmax, 5), 2))
    ax.set_yticklabels(np.round(np.linspace(xmin, xmax, 5), 2), rotation=0, fontsize=10)
    ax.grid(False)
    plt.tight_layout(h_pad=1)
    
    
    #print ('HERE',len(x_orig), k)
    #if (k ==0 or k ==)
    
    for x1 in range(0,len(x_orig),75):
        
        mesh_name = 'visdump_subgrid_%s_mesh.h5'%x1
        #print (mesh_name)
        
        mesh_sg = h5py.File(os.path.join(path_cades,mesh_name),'r')
        mesh_sg_data = list(mesh_sg['0']['Mesh']['Nodes'])
        X_sgmesh,Y_sgmesh,Z_sgmesh = subgrid_mesh(mesh_sg_data)
        
        sg_infile = os.path.join(path_cades,'visdump_subgrid_%s_data.h5'%x1)
        
        ran = subgrid_C(sg_infile,varname_sg='subgrid_%s-total_component_concentration'%x1)[k][::-1]
        ran = [r if r>0 else 1e-12 for r in ran]
        Z_sgmesh = np.arange(0.05,1.05,0.05)#,dtype='int')
        #print ('Ran: ',ran)
        #ax.scatter(x_orig[x1], y_orig[x1],Z_sgmesh, c=ran,cmap=cm.coolwarm,vmin=1e-4,vmax=1)#,norm=matplotlib.colors.LogNorm())
        #ax.scatter(x_orig[x1], y_orig[x1],Z_sgmesh,marker=".",c=ran,cmap=cm.coolwarm,vmin=1e-4,vmax=1,norm=matplotlib.colors.LogNorm())
        sc1 = ax.scatter(x_orig[x1], y_orig[x1],Z_sgmesh,marker=".",c=ran,cmap=cm.coolwarm,vmin=1e-4,vmax=.1,norm=matplotlib.colors.LogNorm())
        #break
        ax.set_zticks([])
        
    
    plt.savefig(path + 'fig_%s.png'%k,dpi=100)
    #break
    plt.close()
    count +=1
    #if count >0:
    #    break

49
49


In [3]:
import re, os

def sorted_nicely( l ):
    convert = lambda text: int(text) if text.isdigit() else text
    alphanum_key = lambda key: [convert(c) for c in re.split('([0-9]+)', key)]
    return sorted(l, key = alphanum_key)

dir_files = os.listdir(path)

Files = sorted_nicely(dir_files)

Files = [f for f in Files if f.endswith('.png')]

#Files = Files[:60] + Files[450:520] #+ Files[37:70]
#Files = Files[:200] + Files[280:] 
img_array = []
for filename in Files[::2]:
    filename = path + filename
    img = cv2.imread(filename)
    height, width, layers = img.shape
    size = (width,height)
    img_array.append(img)
    


In [4]:
out = cv2.VideoWriter(path + 'stream-transport-coweeta.mp4',cv2.VideoWriter_fourcc(*'DIVX'), 3, size)
 
for i in range(len(img_array)):
    out.write(img_array[i])
out.release()

In [ ]:
path='/Users/ajc/Core/PostProcessData/2019/simulations/transport/plots_1014/'
%matplotlib tk
x_orig = X_mesh
y_orig = Y_mesh

centroid = []
thick = [1,1,1]

for k in range(cycles):
    fig, ax = plt.subplots(1,1,figsize=(8,4))
    y1 = y_orig
    y2 = [i + 0.000001 for i in y1]
    X = [x_orig,x_orig]
    Y = [y1,y2]
    
    z = np.ones(len(x_orig))*0

    Z = np.array([z,z])
    data = np.linspace(1,0,len(x_orig))
    Z_data = np.array([data,data])
    
    data_color = cm.jet(Z_data/np.amax(Z_data))
    #ran = Discharge_stream
    ran = Concentration[140]#[k+120]
    ran = [r if r>0 else 1e-12 for r in ran]
    #print (ran)
    sc = ax.scatter(X_mesh,Y_mesh,marker=".",c=ran,cmap=cm.coolwarm,vmin=1e-3,vmax=1,norm=matplotlib.colors.LogNorm())
    #sc = ax.scatter(X_mesh,Y_mesh,marker=".",c=ran,cmap=cm.tab10,vmin=0,vmax=1)# map=plt.cm.get_cmap('prism', 5))
    #plt.colorbar(sc, label='Discharge [cm$^3$/sec]',shrink=.96)#,aspect=10)
    plt.colorbar(sc, label='Concentration [kg/m$^3$]',shrink=.96)#,aspect=10)
    
    xmin, xmax = ax.get_xlim()
    ax.set_xticks(np.round(np.linspace(xmin, xmax, 5), 2))
    ax.set_xticklabels(np.round(np.linspace(xmin, xmax, 5), 2), rotation=45, fontsize=10)
    
    xmin, xmax = ax.get_ylim()
    ax.set_yticks(np.round(np.linspace(xmin, xmax, 5), 2))
    ax.set_yticklabels(np.round(np.linspace(xmin, xmax, 5), 2), rotation=0, fontsize=10)
    ax.grid(False)
    plt.tight_layout(h_pad=1)
   
    #plt.savefig(path + 'figs_%s.png'%k,dpi=500)
    break
    plt.close()
    #break

In [ ]:
#Load all images
"""
import cv2
import numpy as np
import glob
 
img_array = []
for filename in glob.glob(path+'*.png'):
    img = cv2.imread(filename)
    height, width, layers = img.shape
    size = (width,height)
    img_array.append(img)
"""